# 🛍️ Retail Store Sales – Exploratory Data Analysis (EDA)
Este análisis explora un conjunto de datos de ventas de una tienda retail, con el objetivo de descubrir patrones, tendencias y posibles oportunidades de negocio.

#PASO 1: CARGUE Y LIMPIEZA DE DATOS

In [3]:
import pandas as pd
import numpy as np
retail_store_sale=pd.read_csv('retail_store_sales.csv')
print(retail_store_sale.head())

  Transaction_ID Customer_ID       Category          Item  Price_Per_Unit  \
0    TXN_6867343     CUST_09     Patisserie   Item_10_PAT            18.5   
1    TXN_3731986     CUST_22  Milk Products  Item_17_MILK            29.0   
2    TXN_9303719     CUST_02       Butchers   Item_12_BUT            21.5   
3    TXN_9458126     CUST_06      Beverages   Item_16_BEV            27.5   
4    TXN_4575373     CUST_05           Food   Item_6_FOOD            12.5   

   Quantity  Total_Spent  Payment_Method Location Transaction_Date  \
0      10.0        185.0  Digital Wallet   Online       2024-04-08   
1       9.0        261.0  Digital Wallet   Online       2023-07-23   
2       2.0         43.0     Credit Card   Online       2022-10-05   
3       9.0        247.5     Credit Card   Online       2022-05-07   
4       7.0         87.5  Digital Wallet   Online       2022-10-02   

  Discount_Applied  
0             True  
1             True  
2            False  
3              NaN  
4          

In [4]:
print(retail_store_sale.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12575 entries, 0 to 12574
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Transaction_ID    12575 non-null  object 
 1   Customer_ID       12575 non-null  object 
 2   Category          12575 non-null  object 
 3   Item              11362 non-null  object 
 4   Price_Per_Unit    11966 non-null  float64
 5   Quantity          11971 non-null  float64
 6   Total_Spent       11971 non-null  float64
 7   Payment_Method    12575 non-null  object 
 8   Location          12575 non-null  object 
 9   Transaction_Date  12575 non-null  object 
 10  Discount_Applied  8376 non-null   object 
dtypes: float64(3), object(8)
memory usage: 1.1+ MB
None


In [5]:
#We can see that Transaction_Date column is object, we need to change it to datetime format
retail_store_sale['Transaction_Date']=pd.to_datetime(retail_store_sale['Transaction_Date'])

print(retail_store_sale.describe())


       Price_Per_Unit      Quantity   Total_Spent  \
count    11966.000000  11971.000000  11971.000000   
mean        23.365912      5.536380    129.652577   
min          5.000000      1.000000      5.000000   
25%         14.000000      3.000000     51.000000   
50%         23.000000      6.000000    108.500000   
75%         33.500000      8.000000    192.000000   
max         41.000000     10.000000    410.000000   
std         10.743519      2.857883     94.750697   

                    Transaction_Date  
count                          12575  
mean   2023-07-12 20:23:41.105368064  
min              2022-01-01 00:00:00  
25%              2022-09-30 00:00:00  
50%              2023-07-13 00:00:00  
75%              2024-04-24 00:00:00  
max              2025-01-18 00:00:00  
std                              NaN  


In [6]:
#CHECK FOR DUPLICATES
print(retail_store_sale.duplicated().sum())


0


In [7]:
#Standardize text data: Remove white spaces

#Get a list of every object datatype columns.
columns=retail_store_sale.select_dtypes('object').columns.tolist()
print(columns)
#For loop apply strip (Delete white spaces at beginning and end) of all object columns.
for col in columns:
    retail_store_sale[col]=retail_store_sale[col].astype(str).str.strip()



['Transaction_ID', 'Customer_ID', 'Category', 'Item', 'Payment_Method', 'Location', 'Discount_Applied']


NULL VALUES
According to the INFO box, the following columns have NULLs: Item, Price_Per_Unit, Quantity, Total_Spent and Discount_Applied

Category, Item and Price_Per_Unit are related. Is probably that items in the same category dont have the same price, and then, I can fill the missing values.  

In [ ]:
#Create a Copy if I miss something
retail_item=retail_store_sale.copy()
#Replace NaN, nan, NAN strings and empty cells, " " cells, for actual NaN values in all the Database.
retail_item= retail_item.replace(r'^\s*$', np.nan, regex=True)
retail_item= retail_item.replace(['nan', 'NaN', 'NAN'], np.nan)

#Checking that items in the same category dont have the same price
df_clean = retail_item.dropna()
check = df_clean.groupby(['Category', 'Price_Per_Unit'])['Item'].nunique()
#print(check[check > 1])
#As there are only 1 item with the same price in the category, we can match the NaN values of Item and Price_Per_Unit with a table of the correct values
df_correct_flat = df_clean.groupby(['Category', 'Price_Per_Unit'])['Item'].unique().explode().reset_index()  #This line create a dataframe of the unique combinations of category, price and item, item being an array. Explode quits the agrupations and give a singular value to each row. Reset index makes all columns real columns again, due to the group by
#print(df_correct_flat)

#Match the Item / Price_Per_Unit value for the combination of Category and Item / Price_Per_Unit. As it is a merge, the column is added at the end.
item_price=['Item','Price_Per_Unit']
#For that adjust NaN for Item and Price_Per_Unit cols at the same line
for col in item_price:
    #If for the ON argument of the merge
    if col == "Item":
        on_col=['Category','Price_Per_Unit']
    else:
        on_col=['Category','Item']
    # Merge of the correct Item/Price_Per_Unit column
    retail_item=pd.merge(retail_item,df_correct_flat,how='left',on=on_col,suffixes=("","_correct"))
    #Now, we need to fillna values of Item and Price_Per_Unit with the values of "_correct" column
    retail_item[col]=retail_item[col].fillna(retail_item[f'{col}_correct'])
    #Delete the _correct column
    retail_item.drop(columns=[f'{col}_correct'], inplace=True)

#Check if the remaning nulls of Item and Price_Per_Unit are the same

null_itemprice=retail_item[(retail_item['Item'].isnull()) & (retail_item['Price_Per_Unit'].isnull())].isnull().sum()

print(null_itemprice)

Transaction_ID        0
Customer_ID           0
Category              0
Item                609
Price_Per_Unit      609
Quantity              0
Total_Spent           0
Payment_Method        0
Location              0
Transaction_Date      0
Discount_Applied    205
dtype: int64


The remaining NaN in Item and Price Per Unit are in the columns in which the two columns are NaN, so the merge doesnt work.
There are still NaN on those 2 columns, but also in Quantity, Total_Spent and Discount_Applied.
As before between Item, Price_Per_Unit and Quantity, there is a relation between Price_Per_Unit, Quantity and Total_Spent. So we can use a similar logic to aproach this NaN's.

In [33]:
#Create a copy of the retail_item table
retail_quantity=retail_item.copy()
#Total_Spent=Price_Per_Unit * Quantity     -- We can get these 3 variables from this formula:
#FILLING Price_Per_Unit, Quantity and Total_Spent. FOR LOOP runs through the list, in which an if statement performs the correct operation depending on the value of col
nan_cols=['Price_Per_Unit','Quantity','Total_Spent']
for col in nan_cols:
    if col == "Price_Per_Unit":
        retail_quantity[f'Fill_{col}']=retail_quantity['Total_Spent'] / retail_quantity['Quantity']
    elif col == "Quantity":
        retail_quantity[f'Fill_{col}']=retail_quantity['Total_Spent'] / retail_quantity['Price_Per_Unit']
    else:
        retail_quantity[f'Fill_{col}']=retail_quantity['Quantity'] * retail_quantity['Price_Per_Unit']
    #Fill NaN based on the col value.
    retail_quantity[col]=retail_quantity[col].fillna(retail_quantity[f'Fill_{col}'])
    #Drop the created columns.
    retail_quantity.drop(columns=[f'Fill_{col}'], inplace=True)

#At this point, all the NaN of Price_Per_Unit where filled, also the majority of Quantity and Total_Spent, whose remaining NaNs are because there are NaN in both columns in the rows, so the operations return NaN
#As we know now all the Price_Per_Unit, we can fill the remaining NaN of the Item column, by merging the same as in the previous step.

retail_quantity=pd.merge(retail_quantity,df_correct_flat,how='left',on=['Category','Price_Per_Unit'],suffixes=("","_correct"))
#Now, we need to fillna values of Item with the values of "_correct" column
retail_quantity['Item']=retail_quantity['Item'].fillna(retail_quantity['Item_correct'])
#Delete the _correct column
retail_quantity.drop(columns=['Item_correct'], inplace=True)

print(retail_quantity.isnull().sum())

#As I said before, the remaining Quantity and Total_Spent NaN are in rows in which the 2 columns are NaN. There are no relations to fill them
#If the total NaN rows are less than 5% of the total dataframe, i'll delete them. Otherwise I can fill forward or with an aggregation.

print(f'{np.round((retail_quantity['Quantity'].isnull().sum()/len(retail_quantity))*100,2)} %') 

#Is less than 5%, so I can drop those rows, as without the Total_Spent value are not useful for a posterior analysis

retail_quantity.dropna(subset=['Quantity'], inplace=True)
print(retail_quantity.isnull().sum())

#Finally, only Discount_Applied has NaN values. This column is not so useful for analysis, so I can just fill with the mode.

mode_val = retail_quantity['Discount_Applied'].mode(dropna=True)[0]
retail_quantity['Discount_Applied'] = retail_quantity['Discount_Applied'].fillna(mode_val)

#print(retail_quantity['Discount_Applied'].value_counts())

#Now, there are not NaN's in our dataframe, and we can start an EDA.

print(retail_quantity.isnull().sum())


Transaction_ID         0
Customer_ID            0
Category               0
Item                   0
Price_Per_Unit         0
Quantity             604
Total_Spent          604
Payment_Method         0
Location               0
Transaction_Date       0
Discount_Applied    4199
dtype: int64
4.8 %
Transaction_ID         0
Customer_ID            0
Category               0
Item                   0
Price_Per_Unit         0
Quantity               0
Total_Spent            0
Payment_Method         0
Location               0
Transaction_Date       0
Discount_Applied    3988
dtype: int64
Transaction_ID      0
Customer_ID         0
Category            0
Item                0
Price_Per_Unit      0
Quantity            0
Total_Spent         0
Payment_Method      0
Location            0
Transaction_Date    0
Discount_Applied    0
dtype: int64
